In [ ]:
import tensorflow as tf
import pandas as pd

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [ ]:
X_train.shape, X_test.shape

In [ ]:
X_train2 = pd.DataFrame(X_train.reshape((60000, 28 * 28)))
X_test2 = pd.DataFrame(X_test.reshape((10000, 28 * 28)))

In [ ]:
from sklearn.tree import DecisionTreeClassifier

m = DecisionTreeClassifier()
m.fit(X_train2, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, m.predict(X_test2))

In [ ]:
from hilbertcurve.hilbertcurve import HilbertCurve

HilbertCurve(5, 2).points_from_distances(list(range(28)))